In [1]:
import pandas as pd

In [8]:
X= pd.read_csv('orange_large_train.data.chunk1',sep='\t')
y=pd.read_csv('orange_large_train_churn.labels')

/var/folders/6d/ydlrjs7x3_n6lhh19hn27v200000gn/T/ipykernel_23063/1468903315.py:1: DtypeWarning: Columns (14741,14742,14748,14749,14750,14752,14753,14755,14758,14759,14760,14762,14767,14769,14773,14776,14779,14780,14781,14789,14791,14797,14798,14800,14804,14805,14806,14809,14811,14817,14818,14819,14824,14828,14832,14834,14836,14843,14844,14850,14851,14855,14861,14871,14876,14878,14879,14881,14882,14883,14886,14888,14890,14894,14896,14897,14900,14904,14906,14914,14916,14917,14918,14926,14928,14931,14932,14936,14937,14939,14942,14943,14944,14947,14951,14954,14957,14958,14970,14972,14980,14996,14998) have mixed types. Specify dtype option on import or set low_memory=False.
  X= pd.read_csv('orange_large_train.data.chunk1',sep='\t')


In [18]:
X.drop(X.columns[X.isna().any()],axis=1)


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var14875,Var14890,Var14893,Var14903,Var14910,Var14913,Var14921,Var14923,Var14990,Var14993
0,0,0,0,0,0,0,0.0,0,0,0,...,UAc5,5eCd,VicHEoo,_lnU,py8m,JKc4,p3by,eUNaPPAA49Enq,0200,IPyde4c
1,0,0,0,0,0,0,0.0,0,0,0,...,UAc5,zSxe,J7IUB91,_lnU,py8m,2e81,p3by,ILESZvp,X7HG,tQAVcMq
2,0,0,0,0,6,0,0.0,0,0,0,...,UAc5,5eCd,NqRDYUf,_lnU,py8m,z0H8,p3by,ILESZvp,77a1,mnKkkvG
3,0,0,0,0,0,0,0.0,0,0,0,...,UAc5,5eCd,0eyRkLO,MgxIGSo,py8m,mZkv,p3by,2PEs,ABkU,ZF0bafC
4,0,0,0,0,0,0,0.0,0,0,0,...,UAc5,5eCd,_tp5CAA,xfMf,py8m,1zA6,p3by,ED3wDbnc4Y,0200,z1miBkG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0,0,0,0,0,0,0.0,0,0,0,...,UAc5,5eCd,FS6qh7f,_lnU,py8m,zOMd,p3by,ILESZvp,X7HG,zLdRfdX
9995,0,0,0,0,0,0,0.0,0,0,0,...,UAc5,5eCd,c75Jl_a,xfMf,py8m,msXM,p3by,zprlKE91yjo7Z,rRqp,iMHYkhP
9996,0,0,0,0,0,0,0.0,0,0,0,...,UAc5,5eCd,g8PQST5,MgxIGSo,py8m,d0Tk,p3by,2PEs,0200,EBL_HBB
9997,0,0,0,0,0,0,0.0,0,0,0,...,UAc5,5eCd,c75Jl_a,IgsADVV,VFdAxz2nKWJ3b,f2Hp,p3by,tWKPzJ6O138XY,X7HG,iMHYkhP


In [20]:
X.columns[X.isna().any()]

Index(['Var52', 'Var156', 'Var169', 'Var173', 'Var182', 'Var217', 'Var269',
       'Var360', 'Var559', 'Var598',
       ...
       'Var14989', 'Var14991', 'Var14992', 'Var14994', 'Var14995', 'Var14996',
       'Var14997', 'Var14998', 'Var14999', 'Var15000'],
      dtype='object', length=613)